# Classifying Chinese Logs

https://gist.github.com/bartolsthoorn/36c813a4becec1b260392f5353c8b7cc

## Imports

In [289]:
from argparse import Namespace
from collections import Counter
import json
import os
import re
import string
import jieba
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

## Data Vectorization classes

### The Vocabulary

In [290]:
class Vocabulary(object):
    """Class to process text and extract vocabulary for mapping"""

    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existing map of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """

        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
        self._add_unk = add_unk
        self._unk_token = unk_token
        
        self.unk_index = -1
        if add_unk:
            self.unk_index = self.add_token(unk_token) 
        
        
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx, 
                'add_unk': self._add_unk, 
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)

    def add_token(self, token):
        """Update mapping dicts based on the token.

        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """Return the token associated with the index
        
        Args: 
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
            KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

### The Vectorizer

In [340]:
class ReviewVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, review_vocab, rating_vocab):
        """
        Args:
            review_vocab (Vocabulary): maps words to integers
            rating_vocab (Vocabulary): maps class labels to integers
        """
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab

    def vectorize(self, review):
        """Create a collapsed one-hit vector for the review
        
        Args:
            review (str): the review 
        Returns:
            one_hot (np.ndarray): the collapsed one-hot encoding 
        """
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        for token in review.split(" "):
            #if token not in string.punctuation:
            one_hot[self.review_vocab.lookup_token(token)] = 1

        return one_hot

    @classmethod
    def from_dataframe(cls, df, modulenames=[], cutoff=1):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            df (pandas.DataFrame): the review dataset
            cutoff (int): the parameter for frequency-based filtering
        Returns:
            an instance of the ReviewVectorizer
        """
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)
        
        # Add ratings
        for rating in sorted(set(modulenames)):
            rating_vocab.add_token(rating)

        # Add top words if count > provided count
        word_counts = Counter()
        # df = list(map(lambda x: list(x), df))
        # print(df[0])
        for review in df:
            arr = review[0].split(' ')
            for word in arr:
                word_counts[word] += 1
               
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)
  
        print('review_vocab: ', review_vocab, 'rating_vocab: ', rating_vocab)
        return cls(review_vocab, rating_vocab)

    @classmethod
    def from_serializable(cls, contents):
        """Instantiate a ReviewVectorizer from a serializable dictionary
        
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the ReviewVectorizer class
        """
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab =  Vocabulary.from_serializable(contents['rating_vocab'])

        return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)

    def to_serializable(self):
        """Create the serializable dictionary for caching
        
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'review_vocab': self.review_vocab.to_serializable(),
                'rating_vocab': self.rating_vocab.to_serializable()}

In [341]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace(u'\xa0', u' ')
    text = re.sub(r"([，,、&：_|\\+;'!?:。；】【-])", r" ", text)
    seg_list = jieba.cut(text, cut_all=False)
    return " ".join(seg_list)

### The Dataset

In [342]:
def convert_filepath(filepath):
    arr = filepath.split('.')[0].replace('\\', '_').split('_')
    arr = list(filter(lambda x: len(x)>0, arr))
    data = ' '.join(arr)
    return data

class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        Args:
            review_df (pandas.DataFrame): the dataset
            vectorizer (ReviewVectorizer): vectorizer instantiated from dataset
        """
        self.review_df = review_df
        self._vectorizer = vectorizer

        self.train_df = list(filter(lambda x: x[2] =='train', self.review_df))
        self.train_size = len(self.train_df)

        self.val_df = list(filter(lambda x: x[2] =='val', self.review_df))
        self.validation_size = len(self.val_df)

        self.test_df = list(filter(lambda x: x[2] =='test', self.review_df))
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}
        print('55555555', self.validation_size)
        self.set_split('train')

    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_xlsx):
        """Load dataset and make a new vectorizer from scratch
        
        Args:
            review_xlsx (str): location of the dataset
        Returns:
            an instance of ReviewDataset
        """
        df_logs = pd.read_excel(review_xlsx, sheet_name='Sheet2')
        df_logs_modulenames = pd.read_excel(review_xlsx, sheet_name='Sheet1')
        #df = df[(df['MODULENAME'] == 'BR_AAA') | (df['MODULENAME'] == 'NP_HAL')  |
        #        (df['MODULENAME'] == 'RT_NSE')| (df['MODULENAME'] == 'BR_AM')]
        logs = df_logs.to_numpy()
        modulenames = df_logs_modulenames["MODULENAME"].unique()
        print(modulenames)
        newlogs = []
        for i in range(len(logs)):
            one_hot = np.zeros(len(modulenames), dtype=np.float32)
            df_modulenames = df_logs_modulenames[(df_logs_modulenames['CHINESE'] == logs[i][0])]
            for module_index, module_row in df_modulenames.iterrows():
                one_hot[np.where(modulenames == module_row.MODULENAME)[0][0]] = 1
            

            newlogs.append([preprocess_text(logs[i][0]), one_hot, logs[i][1]])

        train_review_df = list(filter(lambda x: x[2] == 'train', newlogs))
        return cls(newlogs, ReviewVectorizer.from_dataframe(train_review_df, modulenames))
    
    @classmethod
    def load_dataset_and_load_vectorizer(cls, review_xlsx, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            review_xlsx (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of ReviewDataset
        """
        
        df = pd.read_excel(review_xlsx)
        df = df[(df['MODULENAME'] == 'BR_AAA') | (df['MODULENAME'] == 'NP_HAL') | (df['MODULENAME'] == 'RT_NSE') | \
                 (df['MODULENAME'] =='RTADAPT_FWD_FRAME') | (df['MODULENAME'] =='NP_HAL')]
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of ReviewVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return ReviewVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe 
        
        Args:
            split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df[index]
        text_vector = \
            self._vectorizer.vectorize(row[0])

        return {'x_data': text_vector,
                'y_target': self._target_df[index][1]}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size  
    
def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu", debug=False):
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

## The Model: ReviewClassifier

In [343]:
class ReviewClassifier(nn.Module):
    """ a simple perceptron based classifier """
    def __init__(self, num_features, num_classes):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(ReviewClassifier, self).__init__()
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=num_features),
            nn.ReLU(),
            nn.Linear(num_features, num_classes)
        )

    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,)
        """
        y_out = self.fc1(x_in)  #.squeeze()
#         if apply_sigmoid:
#             y_out = torch.sigmoid(y_out)

        return y_out

## Training Routine

### Helper functions

In [344]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.

    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better

    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]

        # If loss worsened
        if loss_t >= train_state['early_stopping_best_val']:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(y_pred, y_target):
    # print('y_pred: ', y_pred, 'y_target: ', y_target)
    y_target = y_target.cpu()
    elements = y_pred.detach().numpy()
    check_elements = y_target.detach().numpy()
    n_correct= 0
    for i in range(len(elements)):
        el = elements[i]
        m = el[0]
        ind = 0
        for j in range(len(el)):
            if el[j] > m:
                m = el[j]
                ind = j
        correct = True
        for j in range(len(el)):
            if j == ind:
                if check_elements[i][j] < 0.99:
                    correct = False
                    break
#             else:
#                 if check_elements[i][j] > 0.01:
#                     correct = False
#                     break
        if correct:
            n_correct = n_correct + 1

    return n_correct / len(elements) * 100

#### General utilities

In [345]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

### Settings and some prep work

In [346]:
args = Namespace(
    # Data and Path information
    frequency_cutoff=0,
    model_state_file='model.pth',
    review_xlsx='data/chinese_modulenames.xlsx',
    save_dir='model_storage/',
    vectorizer_file='vectorizer.json',
    # No Model hyper parameters
    # Training hyper parameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=10,
    seed=1337,
    # Runtime options
    catch_keyboard_interrupt=True,
    cuda=True,
    expand_filepaths_to_save_dir=True,
    reload_from_files=False,
)

if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
    print("Expanded filepaths: ")
    print("\t{}".format(args.vectorizer_file))
    print("\t{}".format(args.model_state_file))
    
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

print("Using CUDA: {}".format(args.cuda))

args.device = torch.device("cuda" if args.cuda else "cpu")

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

Expanded filepaths: 
	model_storage/vectorizer.json
	model_storage/model.pth
Using CUDA: False


In [347]:
import os
print(os.getcwd())

D:\Data\worktests


### Initializations

In [335]:
if args.reload_from_files:
    # training from a checkpoint
    print("Loading dataset and vectorizer")
    dataset = ReviewDataset.load_dataset_and_load_vectorizer(args.review_xlsx,
                                                            args.vectorizer_file)
else:
    print("Loading dataset and creating vectorizer")
    # create dataset and vectorizer
    dataset = ReviewDataset.load_dataset_and_make_vectorizer(args.review_xlsx)
    dataset.save_vectorizer(args.vectorizer_file)    
vectorizer = dataset.get_vectorizer()



classifier = ReviewClassifier(num_features=len(vectorizer.review_vocab), num_classes=len(vectorizer.rating_vocab))

Loading dataset and creating vectorizer
['3RD_BIN' '74XX_MPU_DRV' '74XX_MPU_INITRD_ROOTFS' '74XX_MPU_SAL'
 '85XX_ICU_INITRD_ROOTFS' '85XX_MPU_DRV' 'BOARD_OM' 'BOARD_SRV' 'BR_AAA'
 'BR_AGENT' 'BR_AM' 'BR_BTRC' 'BR_DHCP' 'BR_DHCP6A' 'BR_DHCPV6'
 'BR_DIAMETER' 'BR_EAPOL' 'BR_FMSR' 'BR_HAADA' 'BR_HAAP' 'BR_IPC_R'
 'BR_L2CP' 'BR_L2TP' 'BR_LAM' 'BR_ND' 'BR_PFC' 'BR_RADIUS' 'BR_RBP'
 'BR_RMADP' 'BR_RPC_R' 'BR_RUI' 'BR_SOFTGRE' 'BR_SRVCFG' 'BR_TERM' 'BR_UM'
 'BR_USERLOG' 'BR_UTL_COMM' 'BR_V5ADA' 'BR_VSM' 'BR_WEB' 'BRAS_CLOUD'
 'BRAS_FEI' 'BRAS_FEIPKT' 'BRAS_FWD' 'BSP' 'BUILD' 'CANBUS_CMU' 'CGN' 'CI'
 'CINE' 'CLK' 'CLOCK' 'CLOCK_PTN' 'Cloud_safe' 'CLOUD_协议' 'CLOUD_隧道'
 'COMMON_DIAG' 'CR_NOUSE_MOD' 'CR55MPUA_DRV' 'CR56LPU1TB_DIAG'
 'CR56SFU100B_DRV' 'CU COMM' 'CU RES' 'cu trunk' 'CU V5ADA' 'DAP'
 'DATATYPE' 'DISASTER' 'DN_CLUSTER' 'DN_FEI_DOP' 'DN_FEI_LIB' 'DN_FEI_SRV'
 'DN_FEISWAGT_FAB' 'DN_FEISWAGT_LIB' 'DN_OFC' 'DRV_BOX_PTN'
 'DRV_BOX_XTN_LLT' 'DRV_DOX_XTN' 'DRVIER_PTN' 'ECU_DRV' 'ETP' 'ETP_D

### Training loop

In [336]:
classifier = classifier.to(args.device)

loss_func = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                 mode='min', factor=0.5,
                                                 patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm_notebook(desc='training routine', 
                          total=args.num_epochs,
                          position=0)

dataset.set_split('train')
train_bar = tqdm_notebook(desc='split=train',
                          total=dataset.get_num_batches(args.batch_size), 
                          position=1, 
                          leave=True)
dataset.set_split('val')
val_bar = tqdm_notebook(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1, 
                        leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].float())
            # step 3. compute the loss
            # print('555555', y_pred, batch_dict['y_target'].float(), '333333', batch_dict['x_data'].float())
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            #loss = loss_func(torch.unsqueeze(y_pred, 0), torch.unsqueeze(batch_dict['y_target'].float(), 0))
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, 
                                  acc=running_acc, 
                                  epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device,
                                           debug=True)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):

            # compute the output
            y_pred = classifier(x_in=batch_dict['x_data'].float())

            # step 3. compute the loss
            loss = loss_func(y_pred, batch_dict['y_target'].float())
            
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            
            val_bar.set_postfix(loss=running_loss, 
                                acc=running_acc, 
                                epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier,
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        if train_state['stop_early']:
            print('Stopped early!!!!')
            break

    print('Finished')

except KeyboardInterrupt:
    print("Exiting loop")

d:\data\worktests\tensor_jypyter\venv\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


d:\data\worktests\tensor_jypyter\venv\lib\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


d:\data\worktests\tensor_jypyter\venv\lib\site-packages\ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


Finished


In [337]:
for i in vectorizer.rating_vocab._token_to_idx:
    print(i) 

3RD_BIN
74XX_MPU_DRV
74XX_MPU_INITRD_ROOTFS
74XX_MPU_SAL
85XX_ICU_INITRD_ROOTFS
85XX_MPU_DRV
BOARD_OM
BOARD_SRV
BRAS_CLOUD
BRAS_FEI
BRAS_FEIPKT
BRAS_FWD
BR_AAA
BR_AGENT
BR_AM
BR_BTRC
BR_DHCP
BR_DHCP6A
BR_DHCPV6
BR_DIAMETER
BR_EAPOL
BR_FMSR
BR_HAADA
BR_HAAP
BR_IPC_R
BR_L2CP
BR_L2TP
BR_LAM
BR_ND
BR_PFC
BR_RADIUS
BR_RBP
BR_RMADP
BR_RPC_R
BR_RUI
BR_SOFTGRE
BR_SRVCFG
BR_TERM
BR_UM
BR_USERLOG
BR_UTL_COMM
BR_V5ADA
BR_VSM
BR_WEB
BSP
BUILD
CANBUS_CMU
CGN
CI
CINE
CLK
CLOCK
CLOCK_PTN
CLOUD_协议
CLOUD_隧道
COMMON_DIAG
CR55MPUA_DRV
CR56LPU1TB_DIAG
CR56SFU100B_DRV
CR_NOUSE_MOD
CU COMM
CU RES
CU V5ADA
Cloud_safe
DAP
DATATYPE
DISASTER
DN_CLUSTER
DN_FEISWAGT_FAB
DN_FEISWAGT_LIB
DN_FEI_DOP
DN_FEI_LIB
DN_FEI_SRV
DN_OFC
DRVIER_PTN
DRV_BOX_PTN
DRV_BOX_XTN_LLT
DRV_DOX_XTN
ECU_DRV
ETP
ETP_DRV
FEISW_AGT_DUNE
FEI_CFG_2800_NETSTREAM
FEI_DOP_588_MIRROR
FEI_DUNE
FEI_EMDI
FE_EZ
FWD_DRIVER
FWD_DRV_BELLE
FWD_DRV_DUNE
FWD_DRV_HSTM
FWD_DRV_PP
FWD_DRV_PPCPU
Fenix_Tools
HDLC_DRV
HI1213_SFU_DRV
HI161X_LPU_DRV
HQOS_5812_5813


In [338]:
# compute the loss & accuracy on the test set using the best available model

classifier.load_state_dict(torch.load(train_state['model_filename']))
classifier = classifier.to(args.device)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())

    # compute the loss
    # loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss = loss_func(torch.unsqueeze(y_pred, 0), torch.unsqueeze(batch_dict['y_target'].float(), 0))
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # compute the accuracy
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [339]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.014
Test Accuracy: 56.83


### Inference

In [330]:
def predict_rating(review, classifier, vectorizer, decision_threshold=0.5):
    """Predict the rating of a review
    
    Args:
        review (str): the text of the review
        classifier (ReviewClassifier): the trained model
        vectorizer (ReviewVectorizer): the corresponding vectorizer
        decision_threshold (float): The numerical boundary which separates the rating classes
    """
    review = preprocess_text(review)
    
    vectorized_review = torch.tensor(vectorizer.vectorize(review))
    result = classifier(vectorized_review.view(1, -1))
    arr = result.detach().numpy()
    print(arr)
    result = np.where(arr[0] == np.amax(arr[0]))
    return vectorizer.rating_vocab.lookup_index(result[0][0])

In [331]:
test_review = "【NE9000-20 4T网板项目质量加固】迭代4共1个问题：接口板在反复复位"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

[[ -7.8421326    3.0755055  -10.812598    -0.16034517]]
【NE9000-20 4T网板项目质量加固】迭代4共1个问题：接口板在反复复位 -> PRODUCT_INCLUDE


In [332]:
test_review = "display access-user web-auth-server <ipv6-address>没法联想"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

[[ 6.161745 -5.446842 -6.517362 -9.794424]]
display access-user web-auth-server <ipv6-address>没法联想 -> BR_UM


In [312]:
test_review = "np hal fwd felib slave include"

classifier = classifier.cpu()
prediction = predict_rating(test_review, classifier, vectorizer, decision_threshold=0.5)
print("{} -> {}".format(test_review, prediction))

np hal fwd felib slave include -> NP_HAL


### Interpretability

In [252]:
classifier.fc1[2].weight.shape

torch.Size([1, 661])

In [253]:
# Sort weights
fc1_weights = classifier.fc1[2].weight.detach()[0]
_, indices = torch.sort(fc1_weights, dim=0, descending=True)
indices = indices.numpy().tolist()

# Top 20 words
print("Influential words in Positive Reviews:")
print("--------------------------------------")
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))
    
print("====\n\n\n")

# Top 20 negative words
print("Influential words in Negative Reviews:")
print("--------------------------------------")
indices.reverse()
for i in range(20):
    print(vectorizer.review_vocab.lookup_index(indices[i]))

Influential words in Positive Reviews:
--------------------------------------
arith40
pts
1213
tmr
dynclass
aaamib
ptn
hpem
stub
fe1213
ne9000
5812
fe8855
devinfo
protected
adapter
tracelog
cmo
msg
acl5886
====



Influential words in Negative Reviews:
--------------------------------------
feiswadmin
map
fim
encap
ffa
txagt
inner
mbt
faultinfo
ase5896
msgopt
hodp
interface
arith100
se
mq1
feicommon
tm
acbc
pe5512


In [26]:
preprocess_text("迭代四共3个问题 1、br_um存的隧道用户由物理槽号修改为逻辑槽号；2、平滑时 最大用户表id增加隧道用户的范围；3、br_lns和um平滑时的老化统计计数统计不上；")

迭代四共3个问题 1 br um存的隧道用户由物理槽号修改为逻辑槽号 2 平滑时 最大用户表id增加隧道用户的范围 3 br lns和um平滑时的老化统计计数统计不上 


'迭代 四共 3 个 问题   1   br   um 存 的 隧道 用户 由 物理 槽 号 修改 为 逻辑 槽 号   2   平滑 时   最大 用户 表 id 增加 隧道 用户 的 范围   3   br   lns 和 um 平滑 时 的 老化 统计 计数 统计 不上  '

In [170]:
arr = np.array([['qos 适配 cx6600 新 单板', np.array([0., 1., 1., 1.]), 'train'], ['  v1r5c00   cu 分离 支持 cgn   主动 质量 提升   cp 侧 基于 domain 查询 用户 表时   记录 回显 比较 卡顿 。', np.array([1., 1., 0., 0.]), 'train'], ['    v8r9c10   autosave 支持 1mac 多 session 项目 线 提前 介入 测试   ipox   acc 特性   问题 简述   配置 autosave 规格   上线 用户 生成 表项 正常   将 规格 修改 小后   autosave 表项 正常   整机 重启   表项 全部 丢失 。   sdv 测试   v8   bng   autosave 支持 1mac 多 session 项目 迭代 二   定位 时 长   4 小时  ', np.array([1., 0., 0., 1.]), 'train'], ['  v8r11c00   网络 硬件   sr   atn 卡 驱动 开发 项目     迭代 十二   m2k   b 合入 主线 b182 版本 后   不 注册   需要 修改 。', np.array([0., 0., 1., 1.]), 'train'], ['  fim 质量 提升   m2k   pst 表不 下发', np.array([0., 0., 1., 1.]), 'train']])
for el in arr[:,0]:
    print(el)

qos 适配 cx6600 新 单板
  v1r5c00   cu 分离 支持 cgn   主动 质量 提升   cp 侧 基于 domain 查询 用户 表时   记录 回显 比较 卡顿 。
    v8r9c10   autosave 支持 1mac 多 session 项目 线 提前 介入 测试   ipox   acc 特性   问题 简述   配置 autosave 规格   上线 用户 生成 表项 正常   将 规格 修改 小后   autosave 表项 正常   整机 重启   表项 全部 丢失 。   sdv 测试   v8   bng   autosave 支持 1mac 多 session 项目 迭代 二   定位 时 长   4 小时  
  v8r11c00   网络 硬件   sr   atn 卡 驱动 开发 项目     迭代 十二   m2k   b 合入 主线 b182 版本 后   不 注册   需要 修改 。
  fim 质量 提升   m2k   pst 表不 下发


In [163]:
np.array([2,3,1,0])

array([2, 3, 1, 0])

### End